In [1]:
import pandas as pd
import sqlalchemy as sa
import numpy as np

import bokeh.io as bio
from bokeh.plotting import figure as bfig
from bokeh.models import WMTSTileSource, ColumnDataSource, LinearColorMapper, HoverTool
from bokeh.tile_providers import CARTODBPOSITRON

### Generate Data

In [2]:
###### make dataframe from Regents Database

engine = sa.create_engine('sqlite:///db/nycedudata (v1_4).db')
meta = sa.MetaData()

regents_query = "SELECT Schools.dbn, Schools.school_name, Schools.latitude, Schools.longitude, \
                    Years.year as school_year, Regents.exam_name, Demo_Categories.demo_var, Regents.test_num, \
                    Regents.mean_score, Regents.num_lt_65, Regents.num_gt_65, Regents.num_gt_80 \
                FROM Schools \
                    INNER JOIN Regents on Regents.dbn = Schools.dbn \
                    INNER JOIN Demo_Categories ON Demo_Categories.demo_cat_id = Regents.demo_id \
                    INNER JOIN Years ON Years.year_id = Regents.year_id;"

regents_df = pd.read_sql_query(regents_query, engine)

k = 6378137
regents_df['latitude'] = regents_df['latitude'].apply( lambda x: np.log(np.tan((90+float(x)) * np.pi/360.0))*k )
regents_df['longitude'] = regents_df['longitude'].apply( lambda x: float(x) * (k * np.pi/180) )
regents_df['mean_score'] = regents_df['mean_score'].apply( lambda x: float(x) )

int_cols = ['school_year','num_lt_65','test_num','num_gt_65','num_gt_80']
regents_df[int_cols] = regents_df[int_cols].applymap( lambda x: int(x) )

regents_df['pct_lt_65'] = regents_df['num_lt_65']/regents_df['test_num']*100
regents_df['pct_gt_65'] = regents_df['num_gt_65']/regents_df['test_num']*100
regents_df['pct_gt_80'] = regents_df['num_gt_80']/regents_df['test_num']*100

regents_df.head()

,dbn,school_name,latitude,longitude,school_year,exam_name,demo_var,test_num,mean_score,num_lt_65,num_gt_65,num_gt_80,pct_lt_65,pct_gt_65,pct_gt_80
0,01m034,p.s. 034 franklin d. roosevelt,4.972080e+06,-8.234879e+06,2015,living environment,all students,16,77.9,1,15,7,6.250000,93.750000,43.750000
1,01m034,p.s. 034 franklin d. roosevelt,4.972080e+06,-8.234879e+06,2016,living environment,all students,9,74.0,1,8,2,11.111111,88.888889,22.222222
2,01m140,p.s. 140 nathan straus,4.971019e+06,-8.235756e+06,2015,living environment,all students,9,67.4,3,6,0,33.333333,66.666667,0.000000
3,01m140,p.s. 140 nathan straus,4.971019e+06,-8.235756e+06,2016,living environment,all students,15,72.6,2,13,5,13.333333,86.666667,33.333333
4,01m140,p.s. 140 nathan straus,4.971019e+06,-8.235756e+06,2017,living environment,all students,9,64.4,5,4,1,55.555556,44.444444,11.111111


### SET MAPPING PARAMETERS

In [3]:
###### Set search parameters
test_stat = 'pct_gt_65'
demo_var = 'black'
test_name = 'common core algebra'
school_year = 2016

###### Set map parameters
padding = 10

south = np.min(regents_df['latitude']) - padding
north = np.max(regents_df['latitude']) + padding
west = np.min(regents_df['longitude']) - padding
east = np.max(regents_df['longitude']) + padding

spectrum_low = 0
spectrum_high = 100

In [4]:
###### Refine dataframe based on search parameters

source_df = regents_df[regents_df['school_year'] == school_year]
source_df = source_df[source_df['demo_var'] == demo_var]
source_df = source_df[source_df['exam_name'] == test_name]

source_df = source_df[['dbn','school_name','longitude','latitude',test_stat]]

# source_df.head()

data_source = ColumnDataSource(source_df)

In [5]:
###### Create map of data

p = bfig(tools='pan, wheel_zoom', x_range=(west,east), y_range=(south,north), 
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(CARTODBPOSITRON)

color_map = LinearColorMapper(palette='Magma256', low=spectrum_low, high=spectrum_high)

p.circle(x='longitude', y='latitude', source=data_source, fill_color={'field':test_stat, 'transform':color_map}, size=10)
p.add_tools(HoverTool(tooltips=[('DBN','@dbn'),('Name','@school_name'),('Score','@{0}'.format(test_stat))]))
bio.show(p)